# Profiling with galileo-experiments

Galileo-experiments enables users to easily profile/benchmark applications on different hosts.
This task focuses on the simple evaluation of one specific application (i.e., object detection) using one or multiple instances on a single host.

The following section will execute an experiment that:
1. Spawns `no_pods` Pods that contain a Mobilenet Inference OpenFaaS-based function
2. Spawns `n_clients` clients that will each send `n` requests with an interarrival time of `ia`
3. Afterwards, we show how to analyze the data with `galileojp`.

In general, the workflow presented here can be used for all applications that are callable via HTTP.
The following things are required to support other applications (defined in the abstract class `ProfilingApplication`):
1. Implement a `pod_factory` function that takes 3 arguments (`pod_name`, `image`, `resource_requests`)
   and returns a `kubernetes.client.V1Container`.
2. Implement a function that spawns and returns a `ClientGroup`.
   

In [2]:
from galileo.shell.shell import init
from galileo.worker.context import Context
from galileoexperiments.api.model import ProfilingWorkloadConfiguration
from galileoexperiments.experiment.profiling.run import run_profiling_workload

from galileoexperimentsextensions.mobilenet.profiling.app import MobilenetProfilingApplication

### Instantiate galileo context that includes all dependencies needed to execute an experiment

In [3]:
ctx = Context()
rds = ctx.create_redis()
g = init(rds)

### Configure mobilenet specific parameters (i.e., image_url) and define the function name

In [5]:
params = {
    'service': {
        'name': 'mobilenet',
        'image_url': 'https://i.imgur.com/0jx0gP8.png'
    }
}

### Instantiate the Profiling Application that contains methods that spawn a container and return a ClientGroup

In [6]:
mobilenet_profiling_app = MobilenetProfilingApplication()

### Set experiment parameters

In [8]:
creator = 'creator-name'
host = 'test-host'
image = 'edgerun/mobilenet-inference:1.1'
zone = 'zone-test'
master_node = 'master-node'

### Create ProfilingWorkloadConfiguration

In [9]:
workload_config = ProfilingWorkloadConfiguration(
    creator=creator,
    app_name=params['service']['name'],
    host=host,
    image=image,
    master_node=master_node,
    zone=zone,
    context=g,
    params=params,
    profiling_app=mobilenet_profiling_app,
    no_pods=1,
    n=10,
    ia=0.5,
    n_clients=1
)

### Run the profiling workload

In [ ]:
run_profiling_workload(workload_config)